#  The Battle of Neighborhoods: Exploring Montreal Neighborhoods to find the best location for Korean restaurant.

# Applied Data Science Capstone by IBM/Coursera¶

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

Montreal is the second-most populous city in Canada and most populous city in the Canadian province of Quebec. Montreal has the second-largest economy in Canada based on GDP and the largest in Quebec. The city is today an important center of commerce, finance, industry, technology, culture and world affairs. Montreal was referred to as "Canada's Cultural Capital" as well. The city's many cultural communities have given it a distinct local culture.

This project aims at exploring and segmenting the city of Montreal to give the recommendations to potential investors which were looking for the best place to open the Korean restaurant.

## Data <a name="data"></a>

• We will use the Wikipedia data “List of postal codes of Canada: H” for the Metropolitan Montreal (https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_H) to get the information of the postal-codes and boroughs/neighborhoods names. The data from the Wikipedia page will be scrapped and wrangled, cleaned and then read it into a pandas dataframe.

• To get the spatial date (latitude and longitude) we will be using the Open-source geocoding Nominatim from geopy. Latitude and longitude will be added to pandas dataframe.

• We will use Foursquare API to get the common venues in each neighborhood and then use this feature to group the neighborhoods into clusters to determine the best possible fit for the Korean restaurant.

### Data scraping by using the BeautifulSoup package. After that, data will be cleaned and processed for further analyzing.

Let's import the libraries that we're going to use at first:

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [2]:
import re
import csv
my_url=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_H').text
!pip install bs4 
!pip install lxml 
!pip install html5lib
from bs4 import BeautifulSoup
soup=BeautifulSoup(my_url,'html5lib')


We will only process the cells that have an assigned borough/neighborhood (ignore cells which contain "Not assigned").

In [3]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Neighborhood'] = (row.span.text).split(',')[0]
        cell['Neighborhood'] = (row.span.text).split('(')[0]
        table_contents.append(cell)   

In [4]:
# print(table_contents)
dfM=pd.DataFrame(table_contents)

Now let's have a look at the ata we scraped and process these data into pandas dataset.

In [5]:
dfM.head(124)

,PostalCode,Neighborhood
0,H1A,Pointe-aux-Trembles
1,H2A,"Saint-Michel,East"
2,H3A,Downtown Montreal North
3,H4A,Notre-Dame-de-GrâceNortheast
4,H5A,Place Bonaventure
5,H7A,Duvernay-Est
6,H9A,Dollard-des-OrmeauxNorthwest
7,H1B,Montreal East
8,H2B,AhuntsicNorth
9,H3B,Downtown MontrealEast


In [6]:
dfM.shape

(124, 2)

### Data Cleaning

In [7]:
dfM['Neighborhood']=dfM['Neighborhood'].replace({'Downtown Montreal North(McGill University)':'Downtown Montreal North',
                                             'Downtown MontrealSoutheast (Concordia University)':'Downtown Montreal Southeast',
                                             'Akwesasne Region1A0: Akwesasne':'Akwesasne'})
update_dfM = dfM.drop([dfM.index[32], dfM.index[118]])
  
dfM =update_dfM.reset_index(drop=True)

In [8]:
dfM.head(50)

,PostalCode,Neighborhood
0,H1A,Pointe-aux-Trembles
1,H2A,"Saint-Michel,East"
2,H3A,Downtown Montreal North
3,H4A,Notre-Dame-de-GrâceNortheast
4,H5A,Place Bonaventure
5,H7A,Duvernay-Est
6,H9A,Dollard-des-OrmeauxNorthwest
7,H1B,Montreal East
8,H2B,AhuntsicNorth
9,H3B,Downtown MontrealEast


In [9]:
dfM.shape

(122, 2)

Once we have built a dataframe of the postal code of each neighborhood, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood. We will be leveraging on the Geocoder Python package

In [10]:
!pip install pgeocode

In [11]:
import pgeocode
nomi = pgeocode.Nominatim('ca')
lat_log= nomi.query_postal_code("H1A")

In [12]:
latitude = []
longitude = []
for i in range(len(dfM['PostalCode'])):
  coord= nomi.query_postal_code(dfM['PostalCode'].iloc[i])
  latitude.append(coord.latitude)
  longitude.append(coord.longitude)

dfM['Latitude'] = latitude
dfM['Longitude'] = longitude

In [13]:
dfM.head(50)

,PostalCode,Neighborhood,Latitude,Longitude
0,H1A,Pointe-aux-Trembles,45.6753,-73.5016
1,H2A,"Saint-Michel,East",45.5618,-73.5990
2,H3A,Downtown Montreal North,45.5040,-73.5747
3,H4A,Notre-Dame-de-GrâceNortheast,45.4717,-73.6149
4,H5A,Place Bonaventure,45.4992,-73.5646
5,H7A,Duvernay-Est,45.6739,-73.5924
6,H9A,Dollard-des-OrmeauxNorthwest,45.4948,-73.8317
7,H1B,Montreal East,45.6320,-73.5075
8,H2B,AhuntsicNorth,45.5741,-73.6507
9,H3B,Downtown MontrealEast,45.5005,-73.5684


Now we got the longitudes and latitudes of the neighborhoods. There is just one row with missing data, so we will just drop it:

In [14]:
dfM = dfM.dropna(subset=['Latitude', 'Longitude'])

In [15]:
dfM.shape

(121, 4)

## Methodology <a name="methodology"></a>

In this project, we will search for venues using Foursquare API and then cluster our neighborhoods based on common venues via clustering algorithm Kmeans. 

### Exploring and clustering neighborhoods in Montreal

We will install Folium, a great visualization library, and Kmeans for clustering.

In [16]:
# import k-means from clustering stage
!pip install scikit-learn==0.24.1
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

# All requested packages already installed.



Create a map of Metropolitan Montreal

In [17]:
address = 'Montreal, QC'

geolocator = Nominatim(user_agent="montreal_explorer")
location = geolocator.geocode(address)
montreal_lat = location.latitude
montreal_lon  = location.longitude
print('The geograpical coordinate of Montreal are {}, {}.'.format(montreal_lat, montreal_lon))

The geograpical coordinate of Montreal are 45.4972159, -73.6103642.


### We will use Folium, a great visualization library to create a map of Montreal using latitude and longitude values

In [18]:
# create map of Montreal using latitude and longitude values
map_montreal = folium.Map(location=[montreal_lat, montreal_lon], zoom_start=11)     

# add markers to map
for lat, lng, label in zip(dfM['Latitude'], dfM['Longitude'], dfM['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_montreal)  
    
map_montreal

### We will consider the center area of Monreal, downtown and arount area with the higher density of people including tourists. So let's slice the original dataframe and create a new dataframe of the center data.

In [19]:
search_values = ['Montreal','Notre-Dame', 'Côte']
df = dfM[dfM.Neighborhood.str.contains('|'.join(search_values ))]

In [20]:
df = df.reset_index(drop=True)

In [21]:
df.head(50)

,PostalCode,Neighborhood,Latitude,Longitude
0,H3A,Downtown Montreal North,45.5040,-73.5747
1,H4A,Notre-Dame-de-GrâceNortheast,45.4717,-73.6149
2,H1B,Montreal East,45.6320,-73.5075
3,H3B,Downtown MontrealEast,45.5005,-73.5684
4,H4B,Notre-Dame-de-GrâceSouthwest,45.4604,-73.6303
5,H3G,Downtown MontrealSoutheast,45.4987,-73.5793
6,H3H,Downtown MontrealSouthwest,45.5009,-73.5877
7,H3S,Côte-des-NeigesNorth,45.5063,-73.6297
8,H3T,Côte-des-NeigesNortheast,45.5018,-73.6191
9,H3V,Côte-des-NeigesEast,45.4990,-73.6089


In [22]:
df.shape

(17, 4)

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

Define Foursquare Credentials and Version (cells will be hidden).

In [23]:
# The code was removed by Watson Studio for sharing.

### Let's explore the 16th neighborhood in our dataframe

In [24]:
df['Neighborhood'].iloc[15]

'Old Montreal'

In [25]:
# Get the neighborhood's latitude and longitude values
neighborhood_latitude = df['Latitude'].iloc[15] # neighborhood latitude value
neighborhood_longitude = df['Longitude'].iloc[15] # neighborhood longitude value

neighborhood_name = df['Neighborhood'].iloc[15] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Old Montreal are 45.5057, -73.555.


In [26]:
#let's get the top 100 venues that are in TOld Montreal within a radius of 500 meters
LIMIT = 100 
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)


In [27]:
#Send the GET request and examine the results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6096cfac989a3538333bbb3c'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Vieux-Montréal',
  'headerFullLocation': 'Vieux-Montréal, Montreal',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 87,
  'suggestedBounds': {'ne': {'lat': 45.5102000045, 'lng': -73.54859110018515},
   'sw': {'lat': 45.501199995499995, 'lng': -73.56140889981486}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b6a38f5f964a520d8cd2be3',
       'name': 'Vieux-Port de Montréal / Old Port of Montreal (Vieux-Port de Montréal)',
       'location': {'address': 'Promenade du Vieux-Port',
        'lat': 45.50558802322094,
        'lng': -73.55237551863875,
   

### Explore selected Neighborhoods of Montreal

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now we will run the above function on each neighborhood and create a new dataframe called montreal_venues.

In [29]:
montreal_venues = getNearbyVenues(df['Neighborhood'],df['Latitude'],df['Longitude'])

Downtown Montreal North
Notre-Dame-de-GrâceNortheast
Montreal East
Downtown MontrealEast
Notre-Dame-de-GrâceSouthwest
Downtown MontrealSoutheast 
Downtown MontrealSouthwest
Côte-des-NeigesNorth
Côte-des-NeigesNortheast
Côte-des-NeigesEast
Côte Saint-LucEast
Côte-des-NeigesSouthwest
Côte Saint-LucWest
Hampstead / Côte Saint-Luc
Montreal West
Old Montreal
Downtown MontrealNortheast


Let's check the size of the resulting dataframe

In [30]:
montreal_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Downtown Montreal North,45.504,-73.5747,Hôtel Le Germain Montréal - NOW OPEN,45.502524,-73.574383,Hotel
1,Downtown Montreal North,45.504,-73.5747,Empire,45.503952,-73.571840,Sporting Goods Shop
2,Downtown Montreal North,45.504,-73.5747,Café Humble Lion,45.503369,-73.575044,Café
3,Downtown Montreal North,45.504,-73.5747,Mcgill University Park,45.504253,-73.575343,Park
4,Downtown Montreal North,45.504,-73.5747,Vua Sandwichs,45.504301,-73.572346,Sandwich Place


In [31]:
montreal_venues.shape

(497, 7)

In [32]:
# Let's check how many venues were returned for each neighborhood
montreal_venues.groupby("Neighborhood").count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Côte Saint-LucEast,6,6,6,6,6,6
Côte Saint-LucWest,4,4,4,4,4,4
Côte-des-NeigesEast,4,4,4,4,4,4
Côte-des-NeigesNorth,6,6,6,6,6,6
Côte-des-NeigesNortheast,4,4,4,4,4,4
Côte-des-NeigesSouthwest,15,15,15,15,15,15
Downtown Montreal North,39,39,39,39,39,39
Downtown MontrealEast,67,67,67,67,67,67
Downtown MontrealNortheast,100,100,100,100,100,100


In [33]:
print('There are {} uniques categories.'.format(len(montreal_venues['Venue Category'].unique())))

There are 150 uniques categories.


## Analysis <a name="analysis"></a>

### Analyze Each Neighborhood

In [34]:

montreal_onehot = pd.get_dummies(montreal_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
montreal_onehot['Neighborhood'] = montreal_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [montreal_onehot.columns[-1]] + list(montreal_onehot.columns[:-1])
montreal_onehot = montreal_onehot[fixed_columns]

montreal_onehot.head()

,Yoga Studio,Accessories Store,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beer Bar,Big Box Store,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Bus Station,Business Service,Café,Cajun / Creole Restaurant,Candy Store,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Curling Ice,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food & Drink Shop,Food Service,French Restaurant,Furniture / Home Store,Gastropub,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Laser Tag,Lebanese Restaurant,Lingerie Store,Liquor Store,Lounge,Men's Store,Metro Station,Middle Eastern Restaurant,Mongolian Restaurant,Monument / Landmark,Mountain,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pharmacy,Pizza Place,Plaza,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tea Room,Thai Restaurant,Theater,Tunnel,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Downtown Montreal North,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Downtown Montreal North,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Downtown Montreal North,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Downtown Montreal North,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Downtown Montreal North,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [35]:
montreal_onehot.shape

(497, 150)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [36]:
montreal_grouped = montreal_onehot.groupby('Neighborhood').mean().reset_index()
montreal_grouped

,Neighborhood,Yoga Studio,Accessories Store,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beer Bar,Big Box Store,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Bus Station,Business Service,Café,Cajun / Creole Restaurant,Candy Store,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Curling Ice,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food & Drink Shop,Food Service,French Restaurant,Furniture / Home Store,Gastropub,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Laser Tag,Lebanese Restaurant,Lingerie Store,Liquor Store,Lounge,Men's Store,Metro Station,Middle Eastern Restaurant,Mongolian Restaurant,Monument / Landmark,Mountain,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pharmacy,Pizza Place,Plaza,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tea Room,Thai Restaurant,Theater,Tunnel,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Women's Store
0,Côte Saint-LucEast,0.000000,0.00,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.166667,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.00,0.00,0.166667,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.166667,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.166667,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00
1,Côte Saint-LucWest,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.

In [37]:
montreal_grouped.shape

(17, 150)

### Each neighborhood along with the top 5 most common venues

In [38]:

num_top_venues = 5

for hood in montreal_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = montreal_grouped[montreal_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Côte Saint-LucEast----
                  venue  freq
0            Restaurant  0.17
1      Asian Restaurant  0.17
2           Pizza Place  0.17
3  Fast Food Restaurant  0.17
4           Coffee Shop  0.17


----Côte Saint-LucWest----
          venue  freq
0          Pool  0.25
1  Skating Rink  0.25
2      Pharmacy  0.25
3          Park  0.25
4   Yoga Studio  0.00


----Côte-des-NeigesEast----
                venue  freq
0   Convenience Store  0.25
1         Bus Station  0.25
2    Business Service  0.25
3          Hobby Shop  0.25
4  Persian Restaurant  0.00


----Côte-des-NeigesNorth----
               venue  freq
0  Food & Drink Shop  0.17
1               Park  0.17
2      Shopping Mall  0.17
3      Grocery Store  0.17
4   Sushi Restaurant  0.17


----Côte-des-NeigesNortheast----
               venue  freq
0               Café  0.25
1  College Bookstore  0.25
2           Pharmacy  0.25
3      Metro Station  0.25
4        Yoga Studio  0.00


----Côte-des-NeigesSouthwest----
         

In [39]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Let's create the new dataframe and display the top 10 venues for each neighborhood

In [40]:

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = montreal_grouped['Neighborhood']

for ind in np.arange(montreal_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(montreal_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Côte Saint-LucEast,Restaurant,Asian Restaurant,Pizza Place,Fast Food Restaurant,Coffee Shop,Discount Store,Yoga Studio,Performing Arts Venue,Noodle House,Paper / Office Supplies Store
1,Côte Saint-LucWest,Pool,Skating Rink,Pharmacy,Park,Yoga Studio,Pastry Shop,New American Restaurant,Nightclub,Noodle House,Paper / Office Supplies Store
2,Côte-des-NeigesEast,Convenience Store,Bus Station,Business Service,Hobby Shop,Persian Restaurant,Noodle House,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue
3,Côte-des-NeigesNorth,Food & Drink Shop,Park,Shopping Mall,Grocery Store,Sushi Restaurant,Discount Store,New American Restaurant,Nightclub,Noodle House,Paper / Office Supplies Store
4,Côte-des-NeigesNortheast,Café,College Bookstore,Pharmacy,Metro Station,Yoga Studio,Performing Arts Venue,Noodle House,Paper / Office Supplies Store,Park,Pastry Shop


### Cluster Montreal Neighborhoods.

Run k-means to cluster the neighborhoods into 3 clusters

In [41]:
kclusters = 3

montreal_grouped_clustering = montreal_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(montreal_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [42]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [43]:
montreal_merged = df

# merge montreal_grouped with montreal_data to add latitude/longitude for each neighborhood
montreal_merged = montreal_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood', how='right')

montreal_merged.head() 

,PostalCode,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,H4V,Côte Saint-LucEast,45.4671,-73.6487,0,Restaurant,Asian Restaurant,Pizza Place,Fast Food Restaurant,Coffee Shop,Discount Store,Yoga Studio,Performing Arts Venue,Noodle House,Paper / Office Supplies Store
12,H4W,Côte Saint-LucWest,45.4700,-73.6686,0,Pool,Skating Rink,Pharmacy,Park,Yoga Studio,Pastry Shop,New American Restaurant,Nightclub,Noodle House,Paper / Office Supplies Store
9,H3V,Côte-des-NeigesEast,45.4990,-73.6089,0,Convenience Store,Bus Station,Business Service,Hobby Shop,Persian Restaurant,Noodle House,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue
7,H3S,Côte-des-NeigesNorth,45.5063,-73.6297,0,Food & Drink Shop,Park,Shopping Mall,Grocery Store,Sushi Restaurant,Discount Store,New American Restaurant,Nightclub,Noodle House,Paper / Office Supplies Store
8,H3T,Côte-des-NeigesNortheast,45.5018,-73.6191,0,Café,College Bookstore,Pharmacy,Metro Station,Yoga Studio,Performing Arts Venue,Noodle House,Paper / Office Supplies Store,Park,Pastry Shop


### Let's visualize the resulting clusters

In [44]:
# create map
map_clusters = folium.Map(location=[45.4972159, -73.6103642], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(montreal_merged['Latitude'], montreal_merged['Longitude'], montreal_merged['Neighborhood'], montreal_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Results and Discussion <a name="results"></a>

## Examine Clusters

In [45]:
#Cluster 1  red
montreal_merged.loc[montreal_merged['Cluster Labels'] == 0, montreal_merged.columns[[1] + list(range(5, montreal_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Côte Saint-LucEast,Restaurant,Asian Restaurant,Pizza Place,Fast Food Restaurant,Coffee Shop,Discount Store,Yoga Studio,Performing Arts Venue,Noodle House,Paper / Office Supplies Store
12,Côte Saint-LucWest,Pool,Skating Rink,Pharmacy,Park,Yoga Studio,Pastry Shop,New American Restaurant,Nightclub,Noodle House,Paper / Office Supplies Store
9,Côte-des-NeigesEast,Convenience Store,Bus Station,Business Service,Hobby Shop,Persian Restaurant,Noodle House,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue
7,Côte-des-NeigesNorth,Food & Drink Shop,Park,Shopping Mall,Grocery Store,Sushi Restaurant,Discount Store,New American Restaurant,Nightclub,Noodle House,Paper / Office Supplies Store
8,Côte-des-NeigesNortheast,Café,College Bookstore,Pharmacy,Metro Station,Yoga Studio,Performing Arts Venue,Noodle House,Paper / Office Supplies Store,Park,Pastry Shop
11,Côte-des-NeigesSouthwest,Vietnamese Restaurant,Gun Range,Sri Lankan Restaurant,Convenience Store,Donut Shop,Restaurant,Sandwich Place,Fast Food Restaurant,Flower Shop,Gym
0,Downtown Montreal North,Hotel,Sandwich Place,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Pizza Place,Bookstore,Cosmetics Shop,Shopping Mall,Clothing Store
3,Downtown MontrealEast,Coffee Shop,Restaurant,Pizza Place,Sandwich Place,Hotel,Portuguese Restaurant,Café,Thai Restaurant,Burger Joint,Juice Bar
16,Downtown MontrealNortheast,Café,Hotel,French Restaurant,Plaza,Italian Restaurant,Restaurant,Sandwich Place,Asian Restaurant,Performing Arts Venue,Vietnamese Restaurant
5,Downtown MontrealSoutheast,Café,Italian Restaurant,Art Museum,Hotel,Indian Restaurant,Japanese Restaurant,Juice Bar,Clothing Store,Middle Eastern Restaurant,Pizza Place


In [46]:
#Cluster 2  purple
montreal_merged.loc[montreal_merged['Cluster Labels'] == 1, montreal_merged.columns[[1] + list(range(5, montreal_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Downtown MontrealSouthwest,Historic Site,Park,Scenic Lookout,Mountain,Performing Arts Venue,New American Restaurant,Nightclub,Noodle House,Paper / Office Supplies Store,Pastry Shop
13,Hampstead / Côte Saint-Luc,Park,Big Box Store,Food Service,Yoga Studio,Performing Arts Venue,New American Restaurant,Nightclub,Noodle House,Paper / Office Supplies Store,Pastry Shop


In [47]:
#Cluster 3  green
montreal_merged.loc[montreal_merged['Cluster Labels'] == 2, montreal_merged.columns[[1] + list(range(5, montreal_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Montreal East,Warehouse Store,Furniture / Home Store,Yoga Studio,Pastry Shop,Music Venue,New American Restaurant,Nightclub,Noodle House,Paper / Office Supplies Store,Park


Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster. 

According to the characteristics of each clustering, we could now easily and intuitively find our ideal location in the map. When we click on the spot in the map, the neighborhood name is shown on the pop-up label.

Since there are lots of restaurants in neighborhoods from cluster 1 (red circles) as the most common venues, we easily can see that locations from the cluster 2 (purple circles) are not crowded with restaurants at all. Since we have already limited locations by the city center, we will choose between Downtown Montreal Southwest and Hampstead / Côte Saint-Luc.
Downtown Montreal Southwest could be the best one since it includes large green spaces and recreational facilities. That is a popular area and could be the best location to open the new Korean restaurant. 

This concludes our analysis. We have found the best location with low number of restaurants and no Korean restaurants nearby, the zone is close to city center and very popular. 

## Conclusion <a name="conclusion"></a>

Purpose of this project was to find the best location in Montreal, Canada to open a new Korean restaurants. Location should have low number of restaurants in general (particularly Korean or Asian restaurants). 

By using Foursquare API we found the most common venues and then clustered our neighborhoods based on common venues.

The optimal location to open a new restaurant will be offered to stakeholders for the final decision. The other specific characteristics and additional factors can be taken into consideration, but it is not a part of this project.